In [1]:
import os
import numpy as np
import netCDF4 as nc
import pandas as pd

# 3D

In [2]:
# Information of directory
dir_base = r'D:\Study\Datasets\Glory'
lst_variable = ['u', 'v', 't', 's']
lst_key = ['uo', 'vo', 'thetao', 'so']
lat_s = 0
lon_s = 100
depth_e = 2000
img_size = 288

In [3]:
lst_filenames = os.listdir(os.path.join(dir_base, 'u'))
lst_filenames = sorted(lst_filenames, key=lambda x:(int(x[0: 4]), int(x[5: 7]))) # sorted
print(lst_filenames)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Study\\Datasets\\Glory/u'

In [ ]:
filepath_temp = os.path.join(dir_base, lst_variable[0], lst_filenames[0])
# Extract data
dataset_temp = nc.Dataset(filepath_temp, 'r')
depth_temp = dataset_temp['depth'][:].data
lat_temp = dataset_temp['latitude'][:].data
lon_temp = dataset_temp['longitude'][:].data

# Cut out data
idx_s_lat = np.argwhere(lat_temp == lat_s)[0][0]
idx_e_lat = idx_s_lat + img_size
idx_s_lon = np.argwhere(lon_temp == lon_s)[0][0]
idx_e_lon = idx_s_lon + img_size
idx_depth = np.argwhere(depth_temp <= 2000).reshape(-1)
idx_s_depth = idx_depth[0]
idx_e_depth = idx_depth[-1]
#
depth_img = depth_temp[idx_s_depth: idx_e_depth]
lat_img = lat_temp[idx_s_lat: idx_e_lat]
lon_img = lon_temp[idx_s_lon: idx_e_lon]
#
date_test = pd.date_range(start='1993-01-01', end='2020-12-31', freq='D').values
dataset_temp.close()

In [ ]:
# dataset_temp.close()

In [ ]:
filepath_temp = os.path.join(dir_base, lst_variable[0], lst_filenames[0])
# Extract data
dataset_temp = nc.Dataset(filepath_temp, 'r')
data_temp = dataset_temp['uo'][:]
mask_temp = np.argwhere(~data_temp.mask)
print(mask_temp.shape)
print(mask_temp[0, :])
print(data_temp.data[*mask_temp[0, :]])
import matplotlib.pyplot as plt

plt.plot(data_temp.data[:, 0, 0, 0])
dataset_temp.close()

In [ ]:
dir_axis = r'../../data/info_axis'
np.save(os.path.join(dir_axis, 'depth_glory.npy'), depth_temp)
np.save(os.path.join(dir_axis, 'lat_glory.npy'), lat_temp)
np.save(os.path.join(dir_axis, 'lon_glory.npy'), lon_temp)

np.save(os.path.join(dir_axis, 'depth_img.npy'), depth_img)
np.save(os.path.join(dir_axis, 'lat_img.npy'), lat_img)
np.save(os.path.join(dir_axis, 'lon_img.npy'), lon_img)

np.save(os.path.join(dir_axis, 'time.npy'), date_test)

In [ ]:
lst_file_bad = []

In [10]:
dir_save = r'E:\dataset'
start_file = '2020_08.nc'
idx_start = lst_filenames.index(start_file)
# end_file = '2019_09.nc'
# idx_end = lst_filenames.index(end_file)
for filename in lst_filenames[idx_start: ]:
    print(filename + ' >>>')
    for i, variable in enumerate(lst_variable):
        print(variable)
        #
        filepath_temp = os.path.join(dir_base, variable, filename)

        # Extract data
        dataset_temp = nc.Dataset(filepath_temp, 'r')
        depth_temp = dataset_temp['depth'][:]
        lat_temp = dataset_temp['latitude'][:]
        lon_temp = dataset_temp['longitude'][:]

        # Cut out data
        idx_s_lat = np.argwhere(lat_temp == lat_s)[0][0]
        idx_e_lat = idx_s_lat + img_size
        idx_s_lon = np.argwhere(lon_temp == lon_s)[0][0]
        idx_e_lon = idx_s_lon + img_size
        idx_depth = np.argwhere(depth_temp <= 2000).reshape(-1)
        idx_s_depth = idx_depth[0]
        idx_e_depth = idx_depth[-1]

        #
        data_temp = dataset_temp[lst_key[i]][:, idx_s_depth: idx_e_depth, idx_s_lat: idx_e_lat, idx_s_lon: idx_e_lon].data
        mask_temp = dataset_temp[lst_key[i]][:, idx_s_depth: idx_e_depth, idx_s_lat: idx_e_lat, idx_s_lon: idx_e_lon].mask

        #
        time_temp = dataset_temp['time'][:]
        depth_img = depth_temp[idx_s_depth: idx_e_depth]
        datetime_temp = pd.to_datetime(time_temp * (60 * 60), unit='s', origin='1950-01-01 00:00:00').round('H').strftime("%Y-%m-%d").values

        # Close dataset
        dataset_temp.close()

        for d, depth_ in enumerate(depth_img):
            for t, datetime_ in enumerate(datetime_temp):
                #
                dir_sub_data = os.path.join(dir_save, 'data',
                                            variable, 'level{:02d}'.format(d))
                if not os.path.exists(dir_sub_data):
                    os.makedirs(dir_sub_data)

                dir_sub_mask = os.path.join(dir_save, 'mask',
                                            variable, 'level{:02d}'.format(d))
                if not os.path.exists(dir_sub_mask):
                    os.makedirs(dir_sub_mask)

                data_save_temp = data_temp[t, d, :][np.newaxis, :, :]
                mask_save_temp = mask_temp[t, d, :][np.newaxis, :, :]
                print(data_save_temp.shape, mask_save_temp.shape)
                filename_save_temp = datetime_ + '.npy'
                np.save(os.path.join(dir_sub_data, filename_save_temp), data_save_temp)
                np.save(os.path.join(dir_sub_mask, filename_save_temp), mask_save_temp)
                print(filename_save_temp + ' has saved.')

2020_08.nc >>>
u
(1, 288, 288) (1, 288, 288)
2020-08-01.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-02.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-03.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-04.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-05.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-06.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-07.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-08.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-09.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-10.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-11.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-12.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-13.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-14.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-15.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-16.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-17.npy has saved.
(1, 288, 288) (1, 288, 288)
2020-08-18.npy has saved.
(1, 288, 28

In [12]:
# 检验数据完整性
dir_save = r'E:\dataset\data\u\level00'
f_test = os.listdir(dir_save)
f_test = [f[0: 10] for f in f_test]
date_test = pd.date_range(start='1993-01-01', end='2020-12-31', freq='D').strftime("%Y-%m-%d").values
idx_test = np.argwhere(~np.isin(date_test, f_test)).reshape(-1)
print(date_test[idx_test])

[]


In [16]:
len(f_test)

10227